In [ ]:
import os
import shutil
import time
from tqdm import tqdm
import random
# import cv2


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import PIL.Image
from IPython.display import Image
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
path_train = "./imgs_less/train"
classes = [c for c in os.listdir(path_train) if not c.startswith(".")]
classes.sort()
print(classes)

In [ ]:
path_train = "./imgs_less/train"
classes = [c for c in os.listdir(path_train) if not c.startswith(".")]
classes.sort()
print(classes)

In [ ]:
df = pd.read_excel('/data/rkj/Rkj_code/project1/test-240-path.xlsx', sheet_name=0)
image_paths = df.iloc[:, 0]
print(image_paths)

In [ ]:
updated_paths = image_paths.apply(lambda x: x.replace('imgs', 'imgs_less'))
print(updated_paths)

In [ ]:
updated_paths = image_paths.apply(lambda x: x.replace('imgs', 'imgs_less'))
print(updated_paths)

In [ ]:
d = {"img" : [], "class" : []}
for c in classes:
    imgs = [img for img in os.listdir(os.path.join(path_train,c)) if not img.startswith(".")]
    for img in imgs:
        d["img"].append(img)
        if c == "c0":
            d["class"].append("c0")
        elif c == "c1":
            d["class"].append("c1")
        elif c == "c2":    
            d["class"].append("c2")
        elif c == "c3":    
            d["class"].append("c3")
        elif c == "c4":    
            d["class"].append("c4")
        elif c == "c5":    
            d["class"].append("c5")
        elif c == "c6":    
            d["class"].append("c6")
        else:
            
            d["class"].append("c7")
df = pd.DataFrame(d)
ax = sns.countplot(data=df,x="class")
ax.set(title="Classes distribution")
print("Total number of training data :",len(df))

In [ ]:
transform = transforms.Compose([transforms.Resize((400, 400)),
                                 transforms.RandomRotation(10),
                                 transforms.ToTensor()])


In [ ]:
data = datasets.ImageFolder(root = path_train, transform = transform)
targets = np.array(data.targets)  # get labels
print(len(targets))
total_len = len(data)
training_len = int(1.0*total_len)
print(training_len)

In [ ]:
train_idx, test_idx = train_test_split(
    np.arange(len(targets)),
    test_size=0.5,
    stratify=targets,
    random_state=42
)
training_data = Subset(data, train_idx)

In [ ]:
path_teat = '/data/rkj/Rkj_code/project1/test'

In [ ]:
# data = datasets.ImageFolder(root = path_train, transform = transform)
# training_data = data
testing_data = datasets.ImageFolder(root = path_teat, transform = transform)
print(training_data)
print(testing_data)

In [ ]:
# data = datasets.ImageFolder(root = path_train, transform = transform)
# training_data = data
testing_data = datasets.ImageFolder(root = path_teat, transform = transform)
print(training_data)
print(testing_data)

In [ ]:
# data = datasets.ImageFolder(root = path_train, transform = transform)
# training_data = data
testing_data = datasets.ImageFolder(root = path_teat, transform = transform)
print(training_data)
print(testing_data)

In [ ]:
loader,labels = next(iter(train_loader))
print(loader.shape)
print(labels.view(8,8))
plt.figure(figsize=(16,16))
plt.imshow(torchvision.utils.make_grid(loader,nrow=8).permute((1,2,0)))
plt.axis('off')
plt.show()

In [ ]:
loader,labels = next(iter(train_loader))
print(loader.shape)
print(labels.view(8,8))
plt.figure(figsize=(16,16))
plt.imshow(torchvision.utils.make_grid(loader,nrow=8).permute((1,2,0)))
plt.axis('off')
plt.show()

In [ ]:
loader,labels = next(iter(train_loader))
print(loader.shape)
print(labels.view(8,8))
plt.figure(figsize=(16,16))
plt.imshow(torchvision.utils.make_grid(loader,nrow=8).permute((1,2,0)))
plt.axis('off')
plt.show()

In [ ]:
def train_model(model, criterion, optimizer, scheduler, n_epochs = 5):
    
    losses = []
    accuracies = []
    test_accuracies = []
    # set the model to train mode initially
    model.train()
    for epoch in tqdm(range(n_epochs)):
        since = time.time()
        running_loss = 0.0
        running_correct = 0.0
        for data in train_loader:

            # get the inputs and assign them to cuda
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # calculate the loss/acc later
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()

        epoch_duration = time.time()-since
        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100/32*running_correct/len(train_loader)

        print("Epoch %s, duration: %d s, loss: %.4f, acc: %.4f" % (epoch+1, epoch_duration, epoch_loss, epoch_acc))
        
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        
        # switch the model to eval mode to evaluate on test data
        model.eval()
        test_acc = eval_model(model)
        test_accuracies.append(test_acc)
        
        # re-set the model to train mode after validating
        model.train()
        scheduler.step(test_acc)
        since = time.time()
    print('Finished Training')
    return model, losses, accuracies, test_accuracies

In [ ]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            # print(images)
            print(labels)
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_ft(images)
            _, predicted = torch.max(outputs.data, 1)
            
            print(predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * correct / total
    print('Accuracy of the network on the test images: %d %%' % (
        test_acc))
    return test_acc

In [ ]:
# model_ft = models.resnet50(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# 
# model_ft.fc = nn.Linear(num_ftrs, 8) #No. of classes = 10
# model_ft = model_ft.to(device)
model_ft = models.alexnet(pretrained=False)
num_ftrs = model_ft.classifier[-1].in_features
model_ft.classifier[-1] = nn.Linear(num_ftrs, 8)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
lrscheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=20, threshold = 0.9)

In [ ]:
# takes around 5-6 minutes per epoch with GPU
model_ft, training_losses, training_accs, test_accs = train_model(model_ft, criterion, optimizer, lrscheduler, n_epochs=20)

In [ ]:
plt.title('Training losses')
plt.plot(training_losses)
plt.show()

In [ ]:
plt.title('Training Accuracy')
plt.plot(training_accs)
plt.show()

In [ ]:
plt.title('Test Accuracy')
plt.plot(test_accs)
plt.show()

In [ ]:
torch.save(model_ft.state_dict(), "./alexnet_save_88/model-driver")

In [ ]:
model = models.alexnet()
num_ftrs = model_ft.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, 8)

# model = models.resnet50()
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 8)
model.load_state_dict(torch.load("./alexnet_save_88/model-driver"))
model.eval()
model.cuda()

In [ ]:
path_test = "/data/rkj/Rkj_code/project1/data_part_3"
list_img_test = [img for img in os.listdir(path_test) if not img.startswith(".")]
list_img_test.sort()

In [ ]:
file = random.choice(list_img_test)
im_path = os.path.join(path_test,file)
print(im_path)
display(Image(filename=im_path))
with PIL.Image.open(im_path) as im:
    im = transform(im)
    im = im.unsqueeze(0)
    output = model(im.cuda())
    print(output)
    proba = nn.Softmax(dim=1)(output)
    # proba = torch.argmax(output,dim=1)
    proba = [round(float(elem),4) for elem in proba[0]]
    print(proba)
    print("Predicted class:",class_dict[proba.index(max(proba))])
    print("Confidence:",max(proba))
    proba2 = proba.copy()
    proba2[proba2.index(max(proba2))] = 0.
    print("2nd answer:",class_dict[proba2.index(max(proba2))])
    print("Confidence:",max(proba2))

In [ ]:
def true_pred(test_data,model):
    y_true = []
    y_pred = []
    n = len(test_data)
    sum = 0
    with torch.no_grad():
        for x,y in tqdm(test_data):
            x = x.to(device)
            pred = torch.argmax(model(x),dim=1)
            print(y)
            print(pred)
            y_true.extend(list(np.array(y)))
            y_pred.extend(list(np.array(pred.cpu())))
    return y_true,y_pred

In [ ]:
m = confusion_matrix(y_true, y_pred)
# m  = m.astype('float') / m.sum(axis=1)[:, np.newaxis]

In [ ]:
# 创建绘图
plt.figure(figsize=(10, 8))  # 可以调整大小以适应你的数据规模

# 绘制热力图并在每个方块中显示整数
sns.heatmap(m, annot=True, fmt="d", cmap='viridis', cbar=True)

plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()